In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

ModuleNotFoundError: No module named 'keras'

In [0]:
import numpy as np
import matplotlib.pyplot as plt

def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # Summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [3]:
df = pd.read_csv("housing.csv", sep=",")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [4]:
df.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [0]:
df = df.dropna(axis=0)

In [6]:
df.describe(include='all')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<1H OCEAN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9034
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155,NaN
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099,NaN
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000,NaN
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000,NaN
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000,NaN
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000,NaN


In [7]:
corrm = df.corr()
corrm.style.background_gradient(cmap='coolwarm')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
longitude,1,-0.924616,-0.109357,0.0454802,0.069608,0.10027,0.0565128,-0.0155502,-0.0453982
latitude,-0.924616,1,0.0118991,-0.0366668,-0.0669828,-0.108997,-0.0717742,-0.0796263,-0.144638
housing_median_age,-0.109357,0.0118991,1,-0.360628,-0.320451,-0.295787,-0.302768,-0.118278,0.106432
total_rooms,0.0454802,-0.0366668,-0.360628,1,0.93038,0.857281,0.918992,0.197882,0.133294
total_bedrooms,0.069608,-0.0669828,-0.320451,0.93038,1,0.877747,0.979728,-0.00772285,0.0496862
population,0.10027,-0.108997,-0.295787,0.857281,0.877747,1,0.907186,0.00508662,-0.0252997
households,0.0565128,-0.0717742,-0.302768,0.918992,0.979728,0.907186,1,0.0134339,0.0648935
median_income,-0.0155502,-0.0796263,-0.118278,0.197882,-0.00772285,0.00508662,0.0134339,1,0.688355
median_house_value,-0.0453982,-0.144638,0.106432,0.133294,0.0496862,-0.0252997,0.0648935,0.688355,1


In [0]:
df['nroom'] = df['total_rooms'] / df['households']
df['nbed'] = df['total_bedrooms'] / df['households']
df['npeople'] = df['population'] / df['households']
df.drop(['total_rooms', 'total_bedrooms', 'population', 'households'], axis = 1, inplace = True)

In [9]:
corrm = df.corr()
corrm.style.background_gradient(cmap='coolwarm')

,longitude,latitude,housing_median_age,median_income,median_house_value,nroom,nbed,npeople
longitude,1,-0.924616,-0.109357,-0.0155502,-0.0453982,-0.0273068,0.0134016,0.00230388
latitude,-0.924616,1,0.0118991,-0.0796263,-0.144638,0.106423,0.070025,0.00252238
housing_median_age,-0.109357,0.0118991,1,-0.118278,0.106432,-0.153031,-0.0779176,0.0132582
median_income,-0.0155502,-0.0796263,-0.118278,1,0.688355,0.325307,-0.0622989,0.0188943
median_house_value,-0.0453982,-0.144638,0.106432,0.688355,1,0.151344,-0.0467395,-0.0236394
nroom,-0.0273068,0.106423,-0.153031,0.325307,0.151344,1,0.848616,-0.00487253
nbed,0.0134016,0.070025,-0.0779176,-0.0622989,-0.0467395,0.848616,1,-0.00617882
npeople,0.00230388,0.00252238,0.0132582,0.0188943,-0.0236394,-0.00487253,-0.00617882,1


In [0]:
scaler = preprocessing.StandardScaler()
df['longitude'] = scaler.fit_transform(df['longitude'].values.reshape(-1,1))
df['latitude'] = scaler.fit_transform(df['latitude'].values.reshape(-1,1))
df['housing_median_age'] = scaler.fit_transform(df['housing_median_age'].values.reshape(-1,1))
df['median_income'] = scaler.fit_transform(df['median_income'].values.reshape(-1,1))
df['median_house_value'] = scaler.fit_transform(df['median_house_value'].values.reshape(-1,1))


In [11]:
df.describe(include='all')

,longitude,latitude,housing_median_age,median_income,median_house_value,ocean_proximity,nroom,nbed,npeople
count,2.043300e+04,2.043300e+04,2.043300e+04,2.043300e+04,2.043300e+04,20433,20433.000000,20433.000000,20433.000000
unique,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,<1H OCEAN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,9034,NaN,NaN,NaN
mean,-3.337466e-15,6.980827e-16,-7.584704e-16,3.832505e-16,-1.025841e-16,NaN,5.431344,1.097062,3.071533
std,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,NaN,2.482946,0.476104,10.438269
min,-2.385447e+00,-1.447937e+00,-2.194584e+00,-1.775054e+00,-1.662139e+00,NaN,0.846154,0.333333,0.692308
25%,-1.112692e+00,-7.972780e-01,-8.444662e-01,-6.884113e-01,-7.568419e-01,NaN,4.441441,1.006029,2.429032
50%,5.393926e-01,-6.428050e-01,2.913923e-02,-1.762077e-01,-2.353266e-01,NaN,5.230769,1.048889,2.817582
75%,7.789699e-01,9.768213e-01,6.644886e-01,4.595713e-01,5.010323e-01,NaN,6.052381,1.099631,3.281513


In [0]:
df= df[df.ocean_proximity != 'ISLAND']


labenc = preprocessing.LabelEncoder()
y = labenc.fit_transform(df['ocean_proximity'])
num_classes = len(np.unique(y))
X = df.drop(['ocean_proximity'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(1024, input_dim=8, activation='relu'))
model.add(Dense(500,  activation='relu'))
model.add(Dense(200,  activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              9216      
_________________________________________________________________
dense_2 (Dense)              (None, 500)               512500    
_________________________________________________________________
dense_3 (Dense)              (None, 200)               100200    
_________________________________________________________________
dense_4 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_7 (Dense)              (None, 4)           

In [14]:
history = model.fit(X_train, Y_train, epochs=500, batch_size=200, validation_data=(X_test,Y_test))
plot_model_history(history)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 13686 samples, validate on 6742 samples
Epoch 1/500





13686/13686 [==============================] - 10s 697us/step - loss: 0.7669 - acc: 0.7017 - val_loss: 0.5144 - val_acc: 0.7966
Epoch 2/500
13686/13686 [==============================] - 1s 51us/step - loss: 0.4692 - acc: 0.8113 - val_loss: 0.4108 - val_acc: 0.8382
Epoch 3/500
13686/13686 [==============================] - 1s 51us/step - loss: 0.3817 - acc: 0.8471 - val_loss: 0.3357 - val_acc: 0.8641
Epoch 4/500
13686/13686 [==============================] - 1s 49us/step - loss: 0.3470 - acc: 0.8580 - val_loss: 0.3857 - val_acc: 0.8466
Epoch 5/500
13686/13686 [==============================] - 1s 50us/step - loss: 0.3124 - acc: 0.8707 - val_loss: 0.3211 - val_acc: 0.8683
Epoch 6/500
13686/13686 [==============================] - 1s 51us/step - loss: 0.2975 - acc: 0.8778 - val_loss: 0.3277 - val_acc: 0.8631
Epoch 7/500
13686/1

KeyboardInterrupt: ignored